In [2]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("Random")
Pkg.add("StatsBase")
using CSV, DataFrames, Random, StatsBase

cd(@__DIR__); pwd()

integrated = CSV.read("C:\\Users\\zzuge\\train.csv", DataFrame)
# integrated = CSV.read("C:/saved/phase2/integrated.csv", DataFrame)
@assert nrow(integrated) == 6903882
@assert ncol(integrated) == (2 + 29)

Random.seed!(0)

gdf = groupby(integrated, :application_id, sort = false)

bit_train = Bool[]
for df in gdf
    append!(bit_train, fill(rand(isone.([0,1,1,1,1])), nrow(df)))
end
bit_isapl = combine(gdf, :is_applied, :is_applied => sum).is_applied_sum .> 0

# 복수선택을 하는 경우가 생각보다 꽤 많긴하다
[count(combine(gdf, :is_applied, :is_applied => sum).is_applied_sum .== i) for i in 0:10]

idx_train = findall(bit_train)
idx_isapl = findall(bit_isapl)

trn_micro = integrated[   bit_train  .&& bit_isapl,:]
vld_micro = integrated[(.!bit_train) .&& bit_isapl,:]
@assert count(bit_isapl) == (nrow(trn_micro) + nrow(vld_micro))
sum(trn_micro.is_applied) / nrow(trn_micro)

vld_total = integrated[(.!bit_train),:]

# rule = [
#     :is_applied => sum
# ]


intergrated_colnames = names(integrated)
rule = []
for column in Symbol.(intergrated_colnames)
    if column ∈ [:nl_loan_limit]
        push!(rule, column => mean)
    elseif column ∈ [:nl_loan_rate]
        push!(rule, column => median)
    else
        push!(rule, column => maximum)
    end
end

gdf = groupby(integrated[bit_train,:], :application_id, sort = false)
trn_macro = combine(gdf, rule...)
select!(trn_macro, Not(:application_id))
rename!(trn_macro, intergrated_colnames)
gdf = groupby(integrated[.!bit_train,:], :application_id, sort = false)
vld_macro = combine(gdf, rule...)
select!(vld_macro, Not(:application_id))
rename!(vld_macro, intergrated_colnames)
@assert length(unique(integrated.application_id)) == (nrow(trn_macro) + nrow(vld_macro))
sum(trn_macro.is_applied) / nrow(trn_macro)

CSV.write("C:\\Users\\zzuge\\trn_macro.csv", trn_macro)
CSV.write("C:\\Users\\zzuge\\vld_macro.csv", vld_macro)
CSV.write("C:\\Users\\zzuge\\trn_micro.csv", trn_micro)
CSV.write("C:\\Users\\zzuge\\vld_micro.csv", vld_micro)
CSV.write("C:\\Users\\zzuge\\vld_total.csv", vld_total)



   Resolving package versions...
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\zzuge\.julia\environments\v1.8\Project.toml`
  [2913bbd2] + StatsBase v0.33.21
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Manifest.toml`


"C:\\Users\\zzuge\\vld_total.csv"